In [1]:
#pip install xgboost

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# match your subgroups again
sub1 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_1.csv").assign(subgroup = 1) # Complicated Diabetes
sub2 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_2.csv").assign(subgroup = 2) # Cardiac
sub3 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_3.csv").assign(subgroup = 3) # Hepatic and Addiction
sub4 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_4.csv").assign(subgroup = 4) # Uncomplicated Diabetes
sub5 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_5.csv").assign(subgroup = 5) # Young
sub6 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_6.csv").assign(subgroup = 6) # Unknown
sub7 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_7.csv").assign(subgroup = 7) # Cardiopulmonary

In [4]:
all_subgroups = pd.concat([sub1, sub2, sub3, sub4, sub5, sub6, sub7], ignore_index=True)
sub_df = all_subgroups.drop(columns=['subject_id', 'hadm_id', 'rounded_age'])

In [5]:
#all_subgroups[all_subgroups['subject_id'] == 690]

In [6]:
# 'subgroup' is the target column
X = sub_df.drop('subgroup', axis = 1)
y = sub_df['subgroup']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test

,admission_type_encoded,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,...,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,age
27471,1,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,76
18432,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,68
40533,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,75
43154,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,58
14811,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50184,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,72
42323,1,1,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,82
31069,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,76
41925,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,62


In [8]:
# train the XGBoost model
y_train_adj = y_train - 1
y_test_adj = y_test - 1

dtrain = xgb.DMatrix(X_train, label=y_train_adj)
dtest = xgb.DMatrix(X_test, label=y_test_adj)


# Specify parameters for the XGBoost model
params = {
    'max_depth': 6,  # the maximum depth of each tree
    'eta': 0.3,  # learning rate
    'objective': 'multi:softprob',  # learning object
    'num_class': 7
}

In [9]:
# Train the model
num_round = 100  # the number of training iterations
bst = xgb.train(params, dtrain, num_round)

In [10]:
# Predictions for the training set
train_preds = bst.predict(dtrain) # vectors of probabilities of each class
train_best_preds = np.asarray([np.argmax(line) for line in train_preds])
train_accuracy = accuracy_score(y_train, train_best_preds + 1)  # +1 to adjust back to original label scale
print("Train Accuracy: %.2f%%" % (train_accuracy * 100.0))

# Predictions for the testing set
test_preds = bst.predict(dtest)
test_best_preds = np.asarray([np.argmax(line) for line in test_preds])
test_accuracy = accuracy_score(y_test, test_best_preds + 1)  # +1 to adjust back to original label scale
print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))

Train Accuracy: 99.34%
Test Accuracy: 96.44%


In [11]:
# Make probability predictions
probabilities = bst.predict(dtest)

In [12]:
# subject_id = 453's index = 44
patient_probabilities = probabilities[0]
patient_probabilities.round(2)

array([0.  , 0.04, 0.  , 0.95, 0.  , 0.01, 0.  ], dtype=float32)

## tell me the probabilities of being in each group

In [13]:
subject_ids = [
    26494, 1113, 22202, 6702, 12613, 3011, 11167, 25699, 6873, 3868, 17807,
    25111, 8566, 7354, 16847, 6659, 11952, 145, 17797, 17795, 9467, 453
]

In [14]:
subjects_features = all_subgroups[all_subgroups['subject_id'].isin(subject_ids)].drop(['hadm_id', 'rounded_age'], axis=1)

X_subject_features = subjects_features.drop(['subject_id', 'subgroup'], axis = 1)

In [15]:
# Step 4: Convert the extracted features into DMatrix format.
dsubjects = xgb.DMatrix(X_subject_features)

# Step 5: Predict the probabilities using the trained model.
subjects_probabilities = bst.predict(dsubjects)

In [16]:
# Assuming subjects_probabilities contains the probabilities for the subjects
rounded_probabilities_df = pd.DataFrame(subjects_probabilities.round(2), columns=[f'prob_sub{i+1}' for i in range(subjects_probabilities.shape[1])])

# Concatenate the rounded probabilities with the subject features based on index order
# Make sure subjects_features is a DataFrame containing the relevant features and has the same number of rows as rounded_probabilities_df
combined_df = pd.concat([subjects_features.reset_index(drop=True), rounded_probabilities_df], axis=1)
combined_df

,subject_id,admission_type_encoded,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,...,depression,age,subgroup,prob_sub1,prob_sub2,prob_sub3,prob_sub4,prob_sub5,prob_sub6,prob_sub7
0,453,1,1,0,1,0,0,1,0,0,...,0,80,1,1.00,0.00,0.00,0.00,0.0,0.00,0.00
1,1113,1,1,0,0,0,0,1,0,1,...,0,46,1,0.95,0.00,0.04,0.00,0.0,0.01,0.00
2,1113,1,1,0,0,0,0,1,0,1,...,0,46,1,0.95,0.00,0.04,0.00,0.0,0.01,0.00
3,3868,1,0,1,0,0,0,1,0,0,...,0,65,1,0.92,0.00,0.00,0.07,0.0,0.00,0.00
4,22202,1,1,1,0,0,0,1,0,0,...,0,66,1,1.00,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,17795,1,1,1,1,0,0,0,0,0,...,0,59,7,0.00,0.02,0.00,0.00,0.0,0.00,0.97
82,17795,1,1,1,1,0,0,0,0,0,...,0,59,7,0.00,0.02,0.00,0.00,0.0,0.00,0.97
83,17797,1,1,1,1,1,0,0,0,0,...,0,57,7,0.00,0.00,0.00,0.00,0.0,0.00,1.00
84,17797,1,1,1,1,1,0,0,0,0,...,0,57,7,0.00,0.00,0.00,0.00,0.0,0.00,1.00


In [17]:
# save csv
# combined_df.to_csv('../processed_data/subgroup_probabilities.csv', index=False)